In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
def dense_block(x, num_layers, growth_rate):
    """Creates a dense block with multiple dense layers."""
    for _ in range(num_layers):
        # Batch normalization
        x = layers.BatchNormalization()(x)
        # ReLU activation
        x = layers.Activation('relu')(x)
        # Convolutional layer
        x = layers.Conv2D(growth_rate, kernel_size=(3, 3), padding='same')(x)
        # Concatenate the input with the output of each convolutional layer
        x = layers.Concatenate()([x, x])
    return x

In [3]:
def transition_layer(x, compression_factor):
    """Creates a transition layer to reduce the number of channels."""
    num_channels = int(x.shape[-1])
    # Batch normalization
    x = layers.BatchNormalization()(x)
    # ReLU activation
    x = layers.Activation('relu')(x)
    # Convolutional layer for channel reduction
    x = layers.Conv2D(num_channels * compression_factor, kernel_size=(1, 1), padding='same')(x)
    # Average pooling for spatial reduction
    x = layers.AveragePooling2D(pool_size=(2, 2), strides=2)(x)
    return x

In [4]:
def create_densenet(input_shape, num_classes, num_dense_blocks=3, num_layers_per_block=4, growth_rate=32, compression_factor=0.5):
    """Creates a DenseNet model."""
    # Input layer
    inputs = tf.keras.Input(shape=input_shape)
    
    # Initial convolutional layer
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    
    # Dense blocks and transition layers
    for i in range(num_dense_blocks):
        x = dense_block(x, num_layers_per_block, growth_rate)
        if i != num_dense_blocks - 1:
            x = transition_layer(x, compression_factor)
    
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully connected layer
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

In [6]:
# Define input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape for image classification
num_classes = 3  # Example number of classes

# Create the DenseNet model
model = create_densenet(input_shape, num_classes)

In [7]:
# Print the model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 112, 112, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 56, 56, 64)   0           ['conv2d[0][0]']                 
                                                                                              

In [9]:
# Print the model summary in a tree-like format
tf.keras.utils.plot_model(model, to_file='model.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [10]:
# save model
model.save('gfgModel.h5')
print('Model Saved!')

Model Saved!


In [ ]:
# Create the DenseNet model
model = create_densenet(input_shape, num_classes)

# Print the model summary
model.summary()


In [11]:
def residual_block(x, filters, strides=1, use_projection=False):
    """Creates a residual block with skip connection."""
    shortcut = x
    if use_projection:
        # Use projection shortcut to match dimensions
        shortcut = layers.Conv2D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)
    
    # First convolutional layer
    x = layers.Conv2D(filters, kernel_size=3, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # Second convolutional layer
    x = layers.Conv2D(filters, kernel_size=3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    # Add shortcut to the output of the second convolutional layer
    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

In [12]:
def create_resnet(input_shape, num_classes, num_blocks=[2, 2, 2, 2], initial_filters=64):
    """Creates a ResNet model."""
    inputs = tf.keras.Input(shape=input_shape)
    
    # Initial convolutional layer
    x = layers.Conv2D(initial_filters, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
    
    # Residual blocks
    filters = initial_filters
    for i, num_layers in enumerate(num_blocks):
        # First block of each stage with strides=2 for downsampling
        x = residual_block(x, filters=filters, strides=2, use_projection=True)
        for _ in range(1, num_layers):
            # Additional blocks in each stage with strides=1
            x = residual_block(x, filters=filters)
        filters *= 2  # Double the number of filters for the next stage
    
    # Global average pooling
    x = layers.GlobalAveragePooling2D()(x)
    
    # Fully connected layer
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    
    # Output layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

In [13]:
# Define input shape and number of classes
input_shape = (224, 224, 3)  # Example input shape for image classification
num_classes = 10  # Example number of classes

# Create the ResNet model
model = create_resnet(input_shape, num_classes)

In [14]:
# Print the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_15 (Conv2D)             (None, 112, 112, 64  9472        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_14 (BatchN  (None, 112, 112, 64  256        ['conv2d_15[0][0]']              
 ormalization)                  )                                                           

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.vgg19 import VGG19

In [12]:
# help(DenseNet121)
# Load the pre-trained DenseNet121 model
base_model = DenseNet121()
base_model.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_6 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_6[0][0]']       
                                )                                                       

In [13]:
# help(DenseNet121)

In [7]:
from tensorflow.keras.applications.densenet import preprocess_input
# help(preprocess_input) 

"""Preprocessed `numpy.array` or a `tf.Tensor` with type `float32`.
The input pixels values are scaled between 0 and 1 and each channel is normalized with respect to the ImageNet dataset."""

'Preprocessed `numpy.array` or a `tf.Tensor` with type `float32`.\nThe input pixels values are scaled between 0 and 1 and each channel is normalized with respect to the ImageNet dataset.'

In [14]:
# Define the CNN architecture
def create_cnn(input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=input_shape))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    # model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

In [15]:
# Create the CNN model
input_shape = (224, 224, 3)
num_classes = 3
my_model = create_cnn(input_shape, num_classes)

In [16]:
base_model.trainable=False

In [22]:
# Define input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 3

# Create the custom CNN model
# custom_model = create_custom_model(input_shape, num_classes)

# Concatenate the outputs of DenseNet and custom model
densenet_output = base_model.layers[-1].output
custom_output = my_model.layers[-1].output
concatenated_output = layers.Concatenate()([densenet_output, custom_output])

# Define the merged model
merged_model = models.Model(inputs=[base_model.input, my_model.input], outputs=concatenated_output)

# Compile the merged model
merged_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the summary of the merged model
merged_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_6 (ZeroPadding2  (None, 230, 230, 3)  0          ['input_4[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d_6[0][0]']       
                                )                                                             

 conv1/bn (BatchNormalization)  (None, 112, 112, 64  256         ['conv1/conv[0][0]']             
                                )                                                                 
                                                                                                  
 conv1/relu (Activation)        (None, 112, 112, 64  0           ['conv1/bn[0][0]']               
                                )                                                                 
                                                                                                  
 zero_padding2d_7 (ZeroPadding2  (None, 114, 114, 64  0          ['conv1/relu[0][0]']             
 D)                             )                                                                 
                                                                                                  
 pool1 (MaxPooling2D)           (None, 56, 56, 64)   0           ['zero_padding2d_7[0][0]']       
          